In [81]:
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader

trans = transforms.ToTensor()
mnist_train = torchvision.datasets.FashionMNIST(
    root="../data", train=True, transform=trans, download=True)
mnist_test = torchvision.datasets.FashionMNIST(
    root="../data", train=False, transform=trans, download=True)

batch_size = 256
train_iter = DataLoader(mnist_train, batch_size=batch_size, shuffle=True)
test_iter = DataLoader(mnist_test, batch_size=batch_size, shuffle=False)

In [82]:
num_inputs = 784  # 图片的长宽为28 x 28
num_outputs = 10  # 输出类别数

In [83]:
import torch

w = torch.normal(0, 0.01, size=(num_inputs, num_outputs), requires_grad=True)
b = torch.zeros(num_outputs, requires_grad=True)

# q:为什么w的形状是(num_inputs, num_outputs)，而不是(num_inputs, 1)？
# a:因为每个输入样本对应一个输出类别，所以需要为每个类别设置一个权重向量。
# q:那为什么线性回归里面，w的列数是1？
# a:因为线性回归是单输出模型，每个输入样本对应一个连续值，而Softmax回归是多分类模型，每个输入样本对应多个类别的概率分布。
# q:那么，softmax输出的形状是什么样的？
# a:softmax输出的形状是(batch_size, num_outputs)，其中batch_size是输入样本的数量，num_outputs是类别数。
# q:如何理解线性回归输出的形状和softmax输出的形状，尤其是num_outputs列？
# a:线性回归的输出是一个连续值，而softmax的输出是一个概率分布，其中num_outputs列表示每个类别的概率。

In [84]:
def softmax(X):
    X_exp = X.exp()
    partition = X_exp.sum(axis=1, keepdims=True)  # 计算每行的总和，结果是列向量
    return X_exp / partition  # 广播机制

In [85]:
def net(X):
    return softmax(torch.matmul(X.reshape((-1, num_inputs)), w) + b)


#q: 这里的-1表示什么？
#a:-1表示任意大小，即任意的行数，列数都是任意的。
#q: 为什么要reshape？为什么要-1？
#a:因为输入的X是一个多维张量，我们需要将其展平为二维张量，以便进行矩阵乘法。-1表示自动计算行数，列数为num_inputs。
print(mnist_train.data.shape)  # torch.Size([60000, 28, 28])

torch.Size([60000, 28, 28])


In [86]:
def cross_entropy(y_hat, y):
    return -torch.log(y_hat[range(len(y_hat)), y])

# q: y_hat的形状是什么样的？
# a: y_hat的形状是(batch_size, num_outputs)。
# q: y的形状是什么样的？
# a: y的形状是(batch_size,)，表示每个样本的真实标签。
# q: 如何理解y_hat[range(len(y_hat)), y]？
# a: y_hat[range(len(y_hat)), y]表示从y_hat中取出每个样本对应的真实标签的概率值，结果是一个一维张量，形状为(batch_size,)。

In [87]:
# def accuracy(y_hat, y):
#     if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
#         y_hat = y_hat.argmax(axis=1)  # 取出每行的最大值索引，就是预测的类别
#     cmp = y_hat.type(y.dtype) == y  # 将y_hat转换为与y相同的数据类型，然后进行比较
#     return float(cmp.type(y.dtype).sum())

def accuracy(y_hat, y):
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = y_hat.argmax(axis=1)
    # 确保 y 是 Tensor
    if not isinstance(y, torch.Tensor):
        y = torch.tensor(y)
    cmp = y_hat.type(y.dtype) == y
    return float(cmp.type(torch.float).sum().item())

In [88]:
class Accumulator:
    def __init__(self, n):
        self.data = [0.0] * n

    def add(self, *args):
        for i, arg in enumerate(args):
            self.data[i] += arg

    def reset(self):
        self.data = [0.0] * len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [89]:
# def evaluate_accuracy(net, data_iter):
#     if isinstance(net, torch.nn.Module):
#         net.eval()  # 设置为评估模式
#     metric = Accumulator(2)  # 累加器，用于计算正确预测的数量和总样本数
#     for X, y in data_iter:
#         metric.add(accuracy(net(X), y), y.numel())
#     return metric[0] / metric[1]  # 返回正确预测的比例

def evaluate_accuracy(net, data_iter):
    if isinstance(net, torch.nn.Module):
        net.eval()  # 设置为评估模式
    metric = Accumulator(2)  # (正确预测数, 总样本数)
    for X, y in data_iter:
        with torch.no_grad():
            y_hat = net(X)
            acc = accuracy(y_hat, y)
            metric.add(acc, y.numel())  # y 必须是 Tensor
    return metric[0] / metric[1]

# q: 什么是评估模式？
# a: 评估模式是指模型在评估阶段的状态，此时会关闭一些训练时特有的操作，如dropout和batch normalization的训练模式。
# q: 什么是metric = Accumulator(2)？
# a: Accumulator是一个累加器，用于存储多个值，这里我们用它来存储正确预测的数量和总样本数。
# q: Accumulator(2)的2表示什么？
# a: 2表示我们需要存储两个值：正确预测的数量和总样本数。
# q: y.numel()是什么？
# a: y.numel()返回y张量中的元素总数，这里表示当前批次的样本数量。
# q: 什么是isinstance()?
# a: isinstance()是一个内置函数，用于检查一个对象是否是指定的类或数据类型的实例。

In [90]:
def train_epoch_ch3(net, train_iter, loss, updater):
    if isinstance(net, torch.nn.Module):
        net.train()  # 设置为训练模式
    metric = Accumulator(3)  # 累加器，用于计算总损失、正确预测的数量和总样本数
    for X, y in train_iter:
        y_hat = net(X)  # 前向传播
        l = loss(y_hat, y).sum()  # 计算损失
        if isinstance(updater, torch.optim.Optimizer):
            updater.zero_grad()  # 清零梯度
            l.backward()  # 反向传播
            updater.step()  # 更新参数
        else:
            l.sum().backward()  # 手动反向传播
            updater(X.shape[0])  # 更新参数
        metric.add(l.item(), accuracy(y_hat, y), y.numel())  # 累加损失、正确预测数量和样本数
    return metric[0] / metric[2], metric[1] / metric[2]  # 返回平均损失和准确率

In [91]:
def train_ch3(net, train_iter, test_iter, loss, num_epochs, updater):
    for epoch in range(num_epochs):
        train_metrics = train_epoch_ch3(net, train_iter, loss, updater)  # 训练一个epoch
        test_acc = evaluate_accuracy(net, test_iter)  # 在测试集上评估准确率
        print(f'epoch {epoch + 1}, loss {train_metrics[0]:f}, '
              f'train acc {train_metrics[1]:f}, test acc {test_acc:f}')

In [92]:
def sgd(params, lr, batch_size):
    # 小批量随机梯度下降
    with torch.no_grad():  # 禁用梯度计算
        for param in params:
            param -= lr * param.grad / batch_size  # .grad 会储存张量在反向传播中计算的梯度
            param.grad.zero_()  # 清零梯度，pytorch不会自动清零

In [93]:
lr = 0.1


def updater(batch_size):
    return sgd([w, b], lr, batch_size)

In [95]:
X, y = next(iter(train_iter))
print("X shape:", X.shape)
print("y shape:", y.shape)
print("net(X) shape:", net(X).shape)
print("accuracy:", accuracy(net(X), y))

X shape: torch.Size([256, 1, 28, 28])
y shape: torch.Size([256])
net(X) shape: torch.Size([256, 10])
accuracy: 23.0


In [96]:
num_epochs = 10
train_ch3(net, train_iter, test_iter, cross_entropy, num_epochs, updater)

epoch 1, loss 0.816614, train acc 0.748600, test acc 0.793000
epoch 2, loss 0.568907, train acc 0.813833, test acc 0.804400
epoch 3, loss 0.525153, train acc 0.825600, test acc 0.813700
epoch 4, loss 0.501837, train acc 0.831217, test acc 0.818000
epoch 5, loss 0.485123, train acc 0.836467, test acc 0.814200
epoch 6, loss 0.474473, train acc 0.840433, test acc 0.820600
epoch 7, loss 0.464406, train acc 0.842567, test acc 0.826600
epoch 8, loss 0.457551, train acc 0.845550, test acc 0.831100
epoch 9, loss 0.452941, train acc 0.846200, test acc 0.835000
epoch 10, loss 0.446985, train acc 0.848150, test acc 0.834000


In [99]:
'''简洁实现'''
from torch import nn

net = nn.Sequential(nn.Flatten(), nn.Linear(784, 10))


def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, std=0.01)


net.apply(init_weights)  # 初始化权重

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=10, bias=True)
)

In [100]:
loss = nn.CrossEntropyLoss()
trainer = torch.optim.SGD(net.parameters(), lr=0.1)

In [101]:
num_epochs = 10
train_ch3(net, train_iter, test_iter, loss, num_epochs, trainer)

epoch 1, loss 0.003081, train acc 0.749650, test acc 0.762800
epoch 2, loss 0.002233, train acc 0.813350, test acc 0.807100
epoch 3, loss 0.002056, train acc 0.826083, test acc 0.807400
epoch 4, loss 0.001962, train acc 0.831700, test acc 0.823300
epoch 5, loss 0.001899, train acc 0.837117, test acc 0.824500
epoch 6, loss 0.001854, train acc 0.840050, test acc 0.825000
epoch 7, loss 0.001818, train acc 0.842667, test acc 0.828800
epoch 8, loss 0.001795, train acc 0.845383, test acc 0.823000
epoch 9, loss 0.001776, train acc 0.846617, test acc 0.831300
epoch 10, loss 0.001756, train acc 0.847800, test acc 0.822700
